**In case of problems or questions, please first check the list of [Frequently Asked Questions (FAQ)](https://stardist.net/docs/faq.html).**

# Multi class StarDist model


This notebook demonstrates how StarDist can addtionally be used to to classify object instances into a fixed number of different object classes (e.g. cell types, phenotypes, etc).

To use multiclass StarDist one has to provide for every training input image `X` and associated label instance mask `Y` and additional **class dictionary** `cls_dict`, that maps instance ids to a discrete set of class labels `label_id -> (1,..., n_classes)`.

The only difference to normal stardist training is 

* Set the `n_classes` variable to the number of all object classes in the `Config` object 
* Set the `classes` variable in `model.train` to a list of class dictionaries for every image/label pair


In the following we demonstrate this workflow for the case of a synthetic dataset constisting of 2 celltypes of different texture.

In [ ]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import numpy as np
import matplotlib
matplotlib.rcParams["image.interpolation"] = None
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from glob import glob
import json
from tqdm import tqdm
from tifffile import imread
from csbdeep.utils import Path, normalize

from stardist import fill_label_holes, random_label_cmap, calculate_extents, gputools_available
from stardist.matching import matching, matching_dataset
from stardist.models import Config2D, StarDist2D, StarDistData2D
from stardist.utils import mask_to_categorical
from stardist.plot import render_label


np.random.seed(42)
lbl_cmap = random_label_cmap()
lbl_cmap_classes = matplotlib.cm.tab20

def plot_img_label(img, lbl, cls_dict, n_classes=2, img_title="image", lbl_title="label", cls_title="classes",**kwargs):
    c = mask_to_categorical(lbl, n_classes=n_classes, classes = cls_dict)
    res = np.zeros(lbl.shape, np.uint16)
    for i in range(1,c.shape[-1]):
        m = c[...,i]>0
        res[m] = i
    class_img = lbl_cmap_classes(res)
    class_img[...,:3][res==0] = 0 
    class_img[...,-1][res==0] = 1
    
    fig, (ai,al,ac) = plt.subplots(1,3, figsize=(18,7), gridspec_kw=dict(width_ratios=(1.,1,1)))
    im = ai.imshow(img, cmap='gray')
    #fig.colorbar(im, ax = ai)
    ai.set_title(img_title)    
    al.imshow(render_label(lbl, .5*normalize(img), normalize_img=False, alpha_boundary=.8,cmap=lbl_cmap))
    al.set_title(lbl_title)
    ac.imshow(class_img)
    ac.imshow(render_label(res, .5*normalize(img), normalize_img=False,  alpha_boundary=.8, cmap=lbl_cmap_classes))
    ac.set_title(cls_title)
    plt.tight_layout()    
    for a in ai,al,ac:
        a.axis("off")
    return ai,al,ac

In [ ]:
# set the number of object classes
n_classes = 2

In [ ]:
# generate synthetic samples of 2D images, label masks, and class dicts 

def generate_sample(n=256):
    from scipy.ndimage.morphology import distance_transform_edt
    from skimage.segmentation import watershed, relabel_sequential 
    from skimage.morphology import disk, binary_erosion, binary_closing
    from scipy import ndimage as ndi
    m = np.random.randint(10,30)
    center = tuple(np.random.randint(0,n,(2,m)))
    markers = np.zeros((n,n),np.uint16)
    markers[center] = np.random.permutation(np.arange(1,m+1))
    
    dist = distance_transform_edt(1-1*(markers>0)) + 4*ndi.zoom(np.random.uniform(0,1,(n//16,n//16)), (16,16), order=1)
    dist *= np.random.uniform(1,2)
    y = watershed(dist, markers, mask=np.exp(-.1*dist)>.1)
        
    y = relabel_sequential(np.max(np.stack([i*binary_closing(y==i,disk(3)) for i in np.unique(y[y>0])],axis = 0),axis=0))[0]

    ind = np.arange(1,y.max()+1)
    np.random.shuffle(ind)
    c1, c2 = ind[:len(ind)//2], ind[len(ind)//2:]
    m1 = np.isin(y,c1)
    m2 = np.isin(y,c2)    
    
    x =  ndi.filters.gaussian_filter((y>0).astype(np.float32),2)
    
    
    noise1 = ndi.zoom(np.random.uniform(0,1,(n//4,n//4)), (4,4), order=3)
    noise2 = ndi.zoom(np.random.uniform(0,1,(n//16,n//16)), (16,16), order=3)
    x[m1] = .2*(1+np.sin(y[m1]))+noise1[m1]
    x[m2] = .2*(1+np.sin(y[m2]))+noise2[m2]
    x = ndi.filters.gaussian_filter(x,1)+2*ndi.filters.gaussian_filter(x,20)
    gain = 100 
    x = np.random.poisson((1+gain*x).astype(int))/gain
    x += .08*np.random.normal(0,1,x.shape)
    cls = dict((i,1 if i in c1 else 2) for i in range(1,m+1))
    return x, y, cls


In [ ]:
# plot an example 
np.random.seed(42)
x,y,cls_dict = generate_sample()

ax = plot_img_label(x,y,cls_dict, n_classes=n_classes)
for a in ax: a.axis("off");

In [ ]:
# show the cls_dict for this example. {label_id -> class_number}

print(cls_dict)

Generate a synthetic training set of 100 random images, label masks, and class dictionaries

In [ ]:
X, Y, C = tuple(zip(*tuple(generate_sample() for _ in tqdm(range(100)))))

print(len(X), len(Y), len(C))
assert len(X) == len(Y) == len(C)

In [ ]:
n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1]

Normalize images and fill small label holes.

In [ ]:
axis_norm = (0,1)   # normalize channels independently
# axis_norm = (0,1,2) # normalize channels jointly
if n_channel > 1:
    print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))
    sys.stdout.flush()

X = [normalize(x,1,99.8,axis=axis_norm) for x in tqdm(X)]
Y = [fill_label_holes(y) for y in tqdm(Y)]

Split into train and validation datasets.

In [ ]:
assert len(X) > 1, "not enough training data"
rng = np.random.RandomState(42)
ind = rng.permutation(len(X))
n_val = max(1, int(round(0.15 * len(ind))))
ind_train, ind_val = ind[:-n_val], ind[-n_val:]
X_val, Y_val, C_val = [X[i] for i in ind_val]  , [Y[i] for i in ind_val] , [C[i] for i in ind_val]
X_trn, Y_trn, C_trn = [X[i] for i in ind_train], [Y[i] for i in ind_train],  [C[i] for i in ind_train] 
print('number of images: %3d' % len(X))
print('- training:       %3d' % len(X_trn))
print('- validation:     %3d' % len(X_val))

Training data consists of pairs of input image and label instances.

In [ ]:
i = min(8, len(X)-1)
img, lbl, cls = X[i], Y[i], C[i]
assert img.ndim in (2,3)
img = img if (img.ndim==2 or img.shape[-1]==3) else img[...,0]
plot_img_label(img,lbl, cls)
None;

# Configuration

A `StarDist2D` model is specified via a `Config2D` object.

In [ ]:
print(Config2D.__doc__)

### Multiclass Model



* Set the `n_classes` variable to the number of all object classes in the `Config` object 
* Set the `classes` variable in `model.train` to a list of class dictionaries for every image/label pair


In [ ]:
# 32 is a good default choice (see 1_data.ipynb)
n_rays = 32

# Use OpenCL-based computations for data generator during training (requires 'gputools')
use_gpu = False and gputools_available()

# Predict on subsampled grid for increased efficiency and larger field of view
grid = (2,2)

conf = Config2D (
    n_rays       = n_rays,
    grid         = grid,
    use_gpu      = use_gpu,
    n_channel_in = n_channel,
    n_classes    = n_classes   # set the number of object classes
)
print(conf)
vars(conf)

In [ ]:
if use_gpu:
    from csbdeep.utils.tf import limit_gpu_memory
    # adjust as necessary: limit GPU memory to be used by TensorFlow to leave some to OpenCL-based computations
    limit_gpu_memory(0.8)

**Note:** The trained `StarDist2D` model will *not* predict completed shapes for partially visible objects at the image boundary if `train_shape_completion=False` (which is the default option).

In [ ]:
#model = StarDist2D(conf, name='stardist_multiclass_cyclic', basedir='models')
model = StarDist2D(None, name='stardist_multiclass_cyclic', basedir='models')

Check if the neural network has a large enough field of view to see up to the boundary of most objects.

In [ ]:
median_size = calculate_extents(list(Y), np.median)
fov = np.array(model._axes_tile_overlap('YX'))
print(f"median object size:      {median_size}")
print(f"network field of view :  {fov}")
if any(median_size > fov):
    print("WARNING: median object size larger than field of view of the neural network.")

# Data Augmentation

You can define a function/callable that applies augmentation to each batch of the data generator.  
We here use an `augmenter` that applies random rotations, flips, and intensity changes, which are typically sensible for (2D) microscopy images (but you can disable augmentation by setting `augmenter = None`).

In [ ]:
def random_fliprot(img, mask): 
    assert img.ndim >= mask.ndim
    axes = tuple(range(mask.ndim))
    perm = tuple(np.random.permutation(axes))
    img = img.transpose(perm + tuple(range(mask.ndim, img.ndim))) 
    mask = mask.transpose(perm) 
    for ax in axes: 
        if np.random.rand() > 0.5:
            img = np.flip(img, axis=ax)
            mask = np.flip(mask, axis=ax)
    return img, mask 

def random_intensity_change(img):
    img = img*np.random.uniform(0.6,2) + np.random.uniform(-0.2,0.2)
    return img


def augmenter(x, y):
    """Augmentation of a single input/label image pair.
    x is an input image
    y is the corresponding ground-truth label image
    """
    x, y = random_fliprot(x, y)
    x = random_intensity_change(x)
    # add some gaussian noise
    sig = 0.02*np.random.uniform(0,1)
    x = x + sig*np.random.normal(0,1,x.shape)
    return x, y

In [ ]:
# plot some augmented examples
img, lbl, cls = X[0],Y[0], C[0]
plot_img_label(img, lbl, cls, n_classes=n_classes)
for _ in range(3):
    img_aug, lbl_aug = augmenter(img,lbl)
    plot_img_label(img_aug, lbl_aug, cls,  img_title="image augmented", lbl_title="label augmented", n_classes=n_classes)

# Training

We recommend to monitor the progress during training with [TensorBoard](https://www.tensorflow.org/programmers_guide/summaries_and_tensorboard). You can start it in the shell from the current working directory like this:

    $ tensorboard --logdir=.

Then connect to [http://localhost:6006/](http://localhost:6006/) with your browser.


In [ ]:
model.train(X_trn, Y_trn, classes = C_trn, validation_data=(X_val,Y_val, C_val), augmenter=augmenter, epochs=200, steps_per_epoch=200)


# Threshold optimization

While the default values for the probability and non-maximum suppression thresholds already yield good results in many cases, we still recommend to adapt the thresholds to your data. The optimized threshold values are saved to disk and will be automatically loaded with the model.

In [ ]:
model.optimize_thresholds(X_val, Y_val)

# Evaluation and Detection Performance

In [ ]:
# let's predict 

i = 8
label, res = model.predict_instances(X_val[i], n_tiles=model._guess_n_tiles(x))

In [ ]:
# the class object ods is stored in the results dict and corresponds to the label ids in increasing order 

def class_from_res(res):
    cls_dict = dict((i+1,c) for i, c in enumerate(res["class_id"]))
    return cls_dict

print(class_from_res(res))

In [ ]:
plot_img_label(X_val[i],Y_val[i], C_val[i], lbl_title="GT")
plot_img_label(X_val[i],label, class_from_res(res), lbl_title="Pred")

Besides the losses and metrics during training, we can also quantitatively evaluate the actual detection/segmentation performance on the validation data by considering objects in the ground truth to be correctly matched if there are predicted objects with overlap (here [intersection over union (IoU)](https://en.wikipedia.org/wiki/Jaccard_index)) beyond a chosen IoU threshold $\tau$.

The corresponding matching statistics (average overlap, accuracy, recall, precision, etc.) are typically of greater practical relevance than the losses/metrics computed during training (but harder to formulate as a loss function). 
The value of $\tau$ can be between 0 (even slightly overlapping objects count as correctly predicted) and 1 (only pixel-perfectly overlapping objects count) and which $\tau$ to use depends on the needed segmentation precision/application.

Please see `help(matching)` for definitions of the abbreviations used in the evaluation below and see the Wikipedia page on [Sensitivity and specificity](https://en.wikipedia.org/wiki/Sensitivity_and_specificity) for further details.

In [ ]:
from scipy.ndimage import zoom
_,_,a = model.keras_model.predict(X_val[i][np.newaxis,...,np.newaxis])
a = zoom(a[0],(2,2,1))

In [ ]:
ss = slice(15,80),slice(15,80)

plot_img_label(X_val[i][ss],label[ss], class_from_res(res), lbl_title="Pred")

plt.figure(figsize=(10,4))
for k in range(3):
    plt.subplot(1,3,k+1);
    plt.imshow(a[...,k][ss], cmap = "viridis");
    plt.axis("off");

In [ ]:
X_val[i][ss].shape

In [ ]:
plt.imshow(X_val[i][ss])

First predict the labels for all validation images:

In [ ]:
Y_val_pred, res_val_pred = tuple(zip(*[model.predict_instances(x, n_tiles=model._guess_n_tiles(x), show_tile_progress=False)
              for x in tqdm(X_val[:])]))

Plot another GT/prediction example 

In [ ]:
i = 10
plot_img_label(X_val[i],Y_val[i], C_val[i], lbl_title="label GT")
plot_img_label(X_val[i],Y_val_pred[i], class_from_res(res_val_pred[i]), lbl_title="label Pred")

Choose several IoU thresholds $\tau$ that might be of interest and for each compute matching statistics for the validation data.

In [ ]:
taus = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
stats = [matching_dataset(Y_val, Y_val_pred, thresh=t, show_progress=False) for t in tqdm(taus)]

Example: Print all available matching statistics for $\tau=0.5$

In [ ]:
stats[taus.index(0.5)]

Plot the matching statistics and the number of true/false positives/negatives as a function of the IoU threshold $\tau$. 

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize=(15,5))

for m in ('precision', 'recall', 'accuracy', 'f1', 'mean_true_score', 'mean_matched_score', 'panoptic_quality'):
    ax1.plot(taus, [s._asdict()[m] for s in stats], '.-', lw=2, label=m)
ax1.set_xlabel(r'IoU threshold $\tau$')
ax1.set_ylabel('Metric value')
ax1.grid()
ax1.legend()

for m in ('fp', 'tp', 'fn'):
    ax2.plot(taus, [s._asdict()[m] for s in stats], '.-', lw=2, label=m)
ax2.set_xlabel(r'IoU threshold $\tau$')
ax2.set_ylabel('Number #')
ax2.grid()
ax2.legend();